# Basic customizations

In [1]:
class BaseCart:
    def __init__(self, user_id):
        self.user_id = user_id
        self.cart_items = {}
        
    def add_item(self, item_id, item_count=1):
        self.cart_items[item_id] = self.cart_items.get(item_id, 0) + item_count

In [2]:
cart_1 = BaseCart("user_id_1")
cart_1.add_item("item_1")
cart_1.add_item("item_2")
print(cart_1)

In [3]:
class Cart(BaseCart):
    def __repr__(self):
        return f"Cart('{self.user_id}')"
    
    def __str__(self):
        return f"Cart for user_id:{self.user_id} has {len(self.cart_items)} items"

In [4]:
cart_1 = Cart("user_id_1")
cart_1.add_item("item_1")
cart_1.add_item("item_2")
cart_1

Cart('user_id_1')

In [5]:
print(cart_1)
print(f"String formating: {cart_1}")

Cart for user_id:user_id_1 has 2 items
String formating: Cart for user_id:user_id_1 has 2 items


In [6]:
class Cart(BaseCart):
    def __bool__(self):
        return len(self.cart_items) > 0

In [7]:
cart_1 = Cart("user_id_1")
cart_1.add_item("item_id_1")
if cart_1:
    print("Cart contains at least 1 item.")
else:
    print("Cart is empty.")

Cart contains at least 1 item.


# Operator Overloading

In [8]:
class Cart(BaseCart):
    def __gt__(self, other):
        return len(self.cart_items) > len(other.cart_items)

In [9]:
cart_1 = Cart("user_id_1")
cart_2 = Cart("user_id_2")
cart_1.add_item("item_id_1")
cart_2.add_item("item_id_1")
print(cart_1 > cart_2)
cart_1.add_item("item_id_2")
print(cart_1 > cart_2)

False
True


In [10]:
from functools import total_ordering
@total_ordering
class Cart(BaseCart):
    def __gt__(self, other):
        return len(self.cart_items) > len(other.cart_items)
    
    def __eq__(self, other):
        return len(self.cart_items) == len(other.cart_items)

In [11]:
cart_1 = Cart("user_id_1")
cart_2 = Cart("user_id_2")
cart_1.add_item("item_id_1")
cart_2.add_item("item_id_1")
print(cart_1 < cart_2)
print(cart_1 <= cart_2)

False
True


In [12]:
class Cart(BaseCart):    
    def __add__(self, other):
        new_cart = Cart(self.user_id)
        for item_id, item_count in self.cart_items.items():
            new_cart.add_item(item_id, item_count)
            
        for item_id, item_count in other.cart_items.items():
            new_cart.add_item(item_id, item_count)
        return new_cart

In [13]:
cart_1, cart_2 = Cart("user_id_1"), Cart("user_id_2")
cart_1.add_item("item_id_1")
cart_2.add_item("item_id_1")
cart_2.add_item("item_id_2")

cart_3 = cart_1 + cart_2

print(cart_3.cart_items)

{'item_id_1': 2, 'item_id_2': 1}


# Containers

In [14]:
class Cart(BaseCart):
    def __contains__(self, key):
        return key in self.cart_items

In [15]:
cart_1 = Cart("user_id_1")
cart_1.add_item("item_id_1")
print("item_id_1" in cart_1)
print("item_id_2" in cart_1)

True
False


In [16]:
class Cart(BaseCart):
    def __len__(self):
        return sum((value for value in self.cart_items.values()))

In [17]:
cart_1 = Cart("user_id_1")
cart_1.add_item("item_id_1", 2)
cart_1.add_item("item_id_2")
len(cart_1)

3

In [18]:
class UpdatedCart(Cart):    
    def __iter__(self):
        for item_id in self.cart_items:
            yield item_id

In [19]:
cart_1 = UpdatedCart("user_id_1")
cart_1.add_item("item_id_1")
cart_1.add_item("item_id_1")
cart_1.add_item("item_id_2")
for item in cart_1:
    print(item)

item_id_1
item_id_2


In [20]:
class UpdatedCart(Cart):    
    def __getitem__(self, key):
        return self.cart_items[key]

In [21]:
cart_1 = UpdatedCart("user_id_1")
cart_1.add_item("item_id_1")
cart_1.add_item("item_id_1")
cart_1["item_id_1"]

2

# collections.abc

In [22]:
from collections import abc
class UpdatedCart(Cart, abc.Mapping):  
    def __iter__(self):
        for key in self.cart_items:
            yield key
            
    def __len__(self):
        return len(self.cart_items)
    
    def __getitem__(self, key):
        return self.cart_items[key]

In [23]:
cart_1 = UpdatedCart("user_id_1")
cart_1.add_item("item_id_1")
cart_1.add_item("item_id_2")

In [24]:
'item_id_1' in cart_1

True

In [25]:
list(cart_1.keys())

['item_id_1', 'item_id_2']

# Simulating open Context Manager

In [26]:
class FileReader:
    def __init__(self, filename):
        self.filename = filename
        self.file_handle = None

    def __enter__(self):
        self.file_handle = open(self.filename)
        return self.file_handle

    def __exit__(self, exc_type, exc_value, traceback):
        if self.file_handle:
            self.file_handle.close()

#with FileReader("/path/to/file") as file_handle:
#    content = file_handle.read()